## <ins style="color:green">**Scikit-Learn Pipelines**</ins>
- Pipelines chains together multiple steps so that the output of each step is used as input to the next step.
- Pipelines makes it easy to apply the same preprocessing to train and test!
- **Step1** ---->_output_---->input-----> **step2** ---->_output_---->input-----> **step3**

### <ins style="color:red">**Project Using Pipeline**</ins>
#### **Strategy :**
__drop_unnecessary_columns__ ---> __Fill_Missing Values__ ---> __OneHotEncoding__ ---> __Scaling__ ---> __Feature_Selection__ ---> __Train_Model__

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer # fill missing values
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler  # Normalization
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier

In [2]:
df = pd.read_csv('../data/train.csv')
df.head(4)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


In [3]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [4]:
df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True)

In [5]:
df.head(4)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S


In [6]:
X = df.iloc[:, 1:]
Y = df.iloc[:, [0]]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train.head(4)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S


In [8]:
# imputation transformer
trf1 = ColumnTransformer([
    ('impute_age', SimpleImputer(), [2]),  # [2] : index of columns = Age
    ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6]) # [6] : Embarked
], remainder='passthrough')

In [9]:
# One hot encoder
trf2 = ColumnTransformer([
    ('ohe_sex_embarked', OneHotEncoder(sparse=False, handle_unknown='ignore'), [1, 6]) # [1, 6] : Sex, Embarked
], remainder='passthrough')

In [10]:
# Scaling
trf3 = ColumnTransformer([
    ('scale', MinMaxScaler(), slice(0, 10))
])
# slice(0, 10) : Apply MinMaxScaler from column 0 to 9

In [11]:
# feature Selection
trf4 = SelectKBest(score_func=chi2, k=8)
# select best 8 feature columns

In [12]:
# train the model
trf5 = DecisionTreeClassifier()

### Create Pipeline

In [13]:
# Pipeline([Tranformer_name, Object])
pipe = Pipeline([
    ('trf1', trf1),
    ('trf2', trf2),
    ('trf3', trf3),
    ('trf4', trf4),
    ('trf5', trf5)
])

### Pipeline Vs make_pipeline
- Pipeline requires naming of steps, make_pipeline does not.
- (Same applies to ColumnTransformer vs make_column_transformer)

In [14]:
# Alternate syntax
make_pipe = make_pipeline(trf1, trf2, trf3, trf4, trf5)

In [15]:
# train
make_pipe.fit(X_train, y_train)

/home/arjun/anaconda3/envs/Project_01/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7fd3be2bd3a0>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

### **Explore Pipeline**

In [19]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=8, score_func=<function chi2 at 0x7fd3be2bd3a0>),
 'trf5': DecisionTreeClassifier()}

In [20]:
pipe.named_steps['trf1']

ColumnTransformer(remainder='passthrough',
                  transformers=[('impute_age', SimpleImputer(), [2]),
                                ('impute_embarked',
                                 SimpleImputer(strategy='most_frequent'),
                                 [6])])

In [21]:
pipe.named_steps['trf1'].transformers_

[('impute_age', SimpleImputer(), [2]),
 ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6]),
 ('remainder', 'passthrough', [0, 1, 3, 4, 5])]

In [24]:
# SimpleImputer use the mean value for filling the NaN values
pipe.named_steps['trf1'].transformers_[0][1].statistics_

array([29.49884615])

In [ ]:
pipe.named_steps['trf1'].transform

In [16]:
from sklearn import set_config
set_config(display = 'diagram')

In [17]:
y_pred = pipe.predict(X_test)

In [18]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)*100

62.56983240223464